In [1]:
import aocd
import numpy as np
import re
from collections import defaultdict
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2021, day=22)
data = puzzle.input_data.split("\n")
data = [re.match("(on|off) x=(.*)\.\.(.*),y=(.*)\.\.(.*),z=(.*)\.\.(.*)", x).groups() for x in data]
data = [tuple([1 if x[0] == "on" else 0] + list(map(int,x[1:]))) for x in data]

In [2]:
# Part a, just numpy it out and worry about the part b later
space = np.zeros((101, 101, 101))
for inst in data[:20]:
    space[inst[1]+50:inst[2]+51, inst[3]+50:inst[4]+51, inst[5]+50:inst[6]+51] = inst[0]
puzzle.answer_a = len(space[space == 1])

In [3]:
class Box:
    def __init__(self, value, x_min, x_max, y_min, y_max, z_min, z_max):
        self.value = value
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
    
    def overlaps(self, box):
        x_overlaps = max(self.x_min, box.x_min) <= min(self.x_max, box.x_max)
        y_overlaps = max(self.y_min, box.y_min) <= min(self.y_max, box.y_max)
        z_overlaps = max(self.z_min, box.z_min) <= min(self.z_max, box.z_max)
        return x_overlaps and y_overlaps and z_overlaps
    
    def size(self):
        return (self.x_max - self.x_min + 1) * (self.y_max - self.y_min + 1) * (self.z_max - self.z_min + 1)
    
    def divide(self, box):
        if(not self.overlaps(box)):
            return [self]

        # turn self into 0-6 smaller boxes, leave box untouched.
        x_min_inside = self.x_min <= box.x_min and box.x_min <= self.x_max
        x_max_inside = self.x_min <= box.x_max and box.x_max <= self.x_max
        
        y_min_inside = self.y_min <= box.y_min and box.y_min <= self.y_max
        y_max_inside = self.y_min <= box.y_max and box.y_max <= self.y_max
        
        z_min_inside = self.z_min <= box.z_min and box.z_min <= self.z_max
        z_max_inside = self.z_min <= box.z_max and box.z_max <= self.z_max
            
        boxes = []
        
        if(x_min_inside):
            boxes.append(Box(self.value, self.x_min, box.x_min-1, self.y_min, self.y_max, self.z_min, self.z_max))
            self.x_min = box.x_min
        if(x_max_inside):
            boxes.append(Box(self.value, box.x_max + 1, self.x_max, self.y_min, self.y_max, self.z_min, self.z_max))
            self.x_max = box.x_max
        
        if(y_min_inside):
            boxes.append(Box(self.value, self.x_min, self.x_max, self.y_min, box.y_min-1, self.z_min, self.z_max))
            self.y_min = box.y_min
        if(y_max_inside):
            boxes.append(Box(self.value, self.x_min, self.x_max, box.y_max + 1, self.y_max, self.z_min, self.z_max))
            self.y_max = box.y_max
            
        if(z_min_inside):
            boxes.append(Box(self.value, self.x_min, self.x_max, self.y_min, self.y_max, self.z_min, box.z_min-1))
            self.z_min = box.z_min
        if(z_max_inside):
            boxes.append(Box(self.value, self.x_min, self.x_max, self.y_min, self.y_max, box.z_max + 1, self.z_max))
            self.z_max = box.z_max
    
        return boxes

In [4]:
boxes = [Box(*x) for x in data]
non_overlapping = []
for b1 in boxes:
    new_boxes = []
    for b2 in non_overlapping:
        bs = b2.divide(b1)
        new_boxes += bs
    non_overlapping = new_boxes
    if(b1.value):
        non_overlapping.append(b1)

puzzle.answer_b = sum([x.size() for x in non_overlapping])